In [ ]:
from IPython.display import display, HTML
import os
import scanpy as sc
import anndata as ann
import pandas as pd
import ktplotspy as kpy

In [ ]:
cpdb_version = 'v4.1.0'
cpdb_target_dir = os.path.join(os.getcwd(), 'cellphonedb')
db_utils.download_database(cpdb_target_dir, cpdb_version)

In [ ]:
cpdb_file_path = 'cellphonedb/cellphonedb.zip'
meta_file_path = '/raid1/YiDingcheng/ALL_LUAD/Myeloid_Epi_meta.csv'
counts_file_path = '/raid1/YiDingcheng/ALL_LUAD/Myeloid_Epi.h5ad'

In [ ]:
adata = sc.read_h5ad("/raid1/YiDingcheng/ALL_LUAD/Myeloid_Epi.h5ad")
adata

In [ ]:
bdata = adata.raw.copy()

In [ ]:
bdata = bdata.to_adata()

In [ ]:
bdata.shape

In [ ]:
meta = pd.read_csv(meta_file_path)
meta.head()

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
deconvoluted, means, pvalues, significant_means = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,
    meta_file_path = meta_file_path,
    counts_file_path = counts_file_path,
    counts_data = "hgnc_symbol",
    threshold=0.1,
    iterations=1000,
    threads=16,
    debug_seed=42,
    subsampling=False,
    output_path='/raid1/YiDingcheng/ALL_LUAD',
    output_suffix = "Myeloid_Epi",
    )

In [ ]:
kpy.plot_cpdb_heatmap(adata = adata, celltype_key = "cell.type",pvals = pvalues, figsize = (8, 8), title = "Sum of significant interactions")

In [ ]:
means = pd.read_table("/raid1/YiDingcheng/ALL_LUAD/statistical_analysis_means_Myeloid_Epi.txt")
pvalues = pd.read_table("/raid1/YiDingcheng/ALL_LUAD/statistical_analysis_pvalues_Myeloid_Epi.txt")

In [ ]:
p = kpy.plot_cpdb(
    adata = adata,
    cell_type1 = "Highly Resistant",
    cell_type2 = "Macro",
    means = means,
    pvals = pvalues,
    celltype_key = "cell.type",
    genes = ["CXCL14", "CX3CL1"],
    highlight_size = 1,
    figsize = (12, 8)
)
p.save("/raid1/YiDingcheng/ALL_LUAD/cellphonedb_Epi.pdf", format = "pdf", dpi = 120)

In [ ]:
kpy.plot_cpdb_chord(
    adata=adata,
    cell_type1="Resistant",
    cell_type2=".",
    means=means,
    pvals=pvalues,
    deconvoluted=deconvoluted,
    celltype_key="cell.type",
    genes=["CXCL14"],
    figsize=(24, 24),
)

In [ ]:
from cellphonedb.utils import search_utils
search_res = search_utils.search_analysis_results(
    query_cell_types_1 = ["CD4T_Treg_CCR8"],
    query_cell_types_2 = ["CD8T_Tem_GZMK", "CD8T_Trm_ZNF683", "CD8T_Tex_CXCL13"],
    significant_means = significant_means,
    deconvoluted = deconvoluted,
    long_format = True
)
search_res.head(50)